# Halton-MaskGIT 3D Model Training

This notebook installs dependencies and runs the `train_3d_full_optimized.py` script to train the Halton-MaskGIT model on the full ModelNet10 dataset.

## 1. Install Dependencies

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 # Ensure CUDA version matches your SageMaker instance (e.g., cu118, cu121)
!pip install git+https://github.com/openai/CLIP.git
!pip install numpy tqdm tensorboard matplotlib # matplotlib might be needed by modelnet_loader.py

## 2. Verify Dataset Location

Ensure the ModelNet10 dataset is available at `/home/sagemaker-user/ModelNet10`. If you followed the previous steps for downloading and extracting, it should be there.

In [ ]:
import os

dataset_path = "/home/sagemaker-user/ModelNet10"

if os.path.exists(dataset_path) and os.path.isdir(dataset_path):
    print(f"Dataset found at: {dataset_path}")
    # Check for class folders directly inside ModelNet10 or inside a nested ModelNet10 folder
    if os.path.exists(os.path.join(dataset_path, 'bathtub')):
        print(f"Example class 'bathtub' found directly in {dataset_path}. Contents (first 5): {os.listdir(dataset_path)[:5]}")
    elif os.path.exists(os.path.join(dataset_path, 'ModelNet10', 'bathtub')):
        print(f"Example class 'bathtub' found in {os.path.join(dataset_path, 'ModelNet10')}. This means your --dataset path should be {os.path.join(dataset_path, 'ModelNet10')}")
        dataset_path = os.path.join(dataset_path, 'ModelNet10') # Adjust for the training script command
    else:
        print(f"Could not find expected ModelNet10 class structure (e.g., 'bathtub' folder) in {dataset_path} or {os.path.join(dataset_path, 'ModelNet10')}. Please verify the extraction.")
else:
    print(f"ERROR: Dataset root directory NOT found at /home/sagemaker-user/ModelNet10. Please ensure it's downloaded and extracted correctly.")

# Store the potentially adjusted dataset_path for the next cell
%store dataset_path

## 3. Run Training Script

The following command runs the `train_3d_full_optimized.py` script.
- Make sure `train_3d_full_optimized.py` and its required modules (`Network`, `Losses`, `datasets` directories) are in the same directory as this notebook.
- The `--dataset` argument below will use the path verified in the cell above.

**Key arguments used:**
- `--output_dir` & `--log_dir`: Set for notebook outputs.
- `--epochs`: Set to 100 for a full run. You can reduce this for a quicker test.
- `--batch_size`: Set to 16 (optimized for ml.g4dn.2xlarge). Adjust if needed.
- `--num_workers`: Reduced to 2. Adjust based on your instance's capabilities.
- `--use_ulip2`: Enables CLIP for text conditioning.
- `--fp16`: Enables mixed-precision training.

In [ ]:
# Retrieve the verified dataset_path
%store -r dataset_path

# Ensure dataset_path is defined (e.g., if running this cell independently)
if 'dataset_path' not in locals():
    dataset_path = "/home/sagemaker-user/ModelNet10" # Default if not stored
    print(f"Using default dataset_path: {dataset_path}. Run previous cell to verify.")

# Construct the command. This assumes train_3d_full_optimized.py is in the current directory.
command = f"""python train_3d_full_optimized.py \
    --dataset '{dataset_path}' \
    --output_dir './outputs/notebook_modelnet_full' \
    --log_dir './logs/notebook_modelnet_full' \
    --epochs 100 \
    --batch_size 16 \
    --num_workers 2 \
    --use_ulip2 \
    --fp16 \
    --masking_strategy 'halton' \
    --mask_ratio 0.65 \
    --progressive_masking \
    --tcga_ratio 0.5 \
    --class_weights \
    --augmentation \
    --save_freq 5 \
    --val_freq 1 \
    --log_freq 10 \
    --early_stopping_patience 15 \
    --seed 42"""
    # --compile_model # Uncomment if you want to try torch.compile
    # --resume_checkpoint './outputs/notebook_modelnet_full/checkpoint_epoch_X.pth' # Uncomment to resume

print(f"Running command:
{command}
")
!{command}

## 4. Monitor with TensorBoard (Optional)

After training starts (or to view previous logs), you can launch TensorBoard. You might need to run this in a new terminal or a separate cell after the training cell has started producing logs.

In [ ]:
# %load_ext tensorboard
# Make sure the logdir matches the --log_dir used in the training command
# %tensorboard --logdir ./logs/notebook_modelnet_full